In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
# from langchain.document_loaders import WebBaseLoader
# from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from pymongo import MongoClient
import params

In [10]:
# Step 1: Load
loaders = [
 PyPDFLoader("Code_de_la_commande_publique.pdf"),
 PyPDFLoader("Vade-mecum_complet.pdf")
]

In [11]:

# Dictionnaire des liens Google Drive
pdf_links = {
    "Code_de_la_commande_publique.pdf": "https://drive.google.com/file/d/1YnpoJhBJ-RkKxt1o2BXQFgbWSkOrPNSJ/view?usp=sharing",
    "Vade-mecum_complet.pdf": "https://drive.google.com/file/d/1eIgLDBh8XqHGHrVRITPNqwuaWH_miKZ_/view?usp=sharing"
}

In [12]:
data = []
for loader in loaders:
    data.extend(loader.load())

In [13]:
# Step 2: Transform (Split)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[
                                               "\n\n", "\n", "(?<=\. )", " "], length_function=len)
docs = text_splitter.split_documents(data)
print('Split into ' + str(len(docs)) + ' docs')

Split into 3494 docs


In [14]:
# Step 3: Embed
# https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.openai.OpenAIEmbeddings.html
embeddings = OpenAIEmbeddings(openai_api_key=params.openai_api_key)

c:\Users\teren\Documents\GitHub\atlas-langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [15]:

# Step 4: Store
# Initialize MongoDB python client
client = MongoClient(params.mongodb_conn_string)
collection = client[params.db_name][params.collection_name]

In [16]:
# Reset w/out deleting the Search Index 
collection.delete_many({})

DeleteResult({'n': 2, 'electionId': ObjectId('7fffffff000000000000015e'), 'opTime': {'ts': Timestamp(1706876663, 78), 't': 350}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1706876663, 79), 'signature': {'hash': b"U0;\xdb\xd1'oU\x1aaU\xf2dy\xe9\x1e\n\xa6\xb6\xc4", 'keyId': 7284123895311892482}}, 'operationTime': Timestamp(1706876663, 78)}, acknowledged=True)

In [17]:
# Insert the documents in MongoDB Atlas with their embedding
# https://github.com/hwchase17/langchain/blob/master/langchain/vectorstores/mongodb_atlas.py
docsearch = MongoDBAtlasVectorSearch.from_documents(
    docs, embeddings, collection=collection, index_name=params.index_name
)